In [3]:
import pandas as pd

shouldnt I put the teamkills into the thing ? 

In [4]:
game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')

unique_games = game_metadata['gameid'].size
print(f'Amount of unique games: {unique_games}')
print(50*'-')

print('Types of values in each dataframe:')
print('Game_metadata:')
print(game_metadata.dtypes)

print('')

print('game_player_stats')
print(game_player_stats.dtypes)
print(50*'-')

Amount of unique games: 74105
--------------------------------------------------
Types of values in each dataframe:
Game_metadata:
gameid         object
date           object
league         object
playoffs        int64
patch         float64
gamelength      int64
dtype: object

game_player_stats
gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
dtype: object
--------------------------------------------------


In [18]:
game_player_stats['teamkills'] = game_player_stats.groupby(['gameid', 'side'])['kills'].transform('sum')
game_player_stats['teamvisionscore'] = game_player_stats.groupby(['gameid', 'side'])['visionscore'].transform('sum')
game_player_stats['teamdeaths'] = game_player_stats.groupby(['gameid', 'side'])['deaths'].transform('sum')
print(game_player_stats.dtypes)

gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
teamkills              int64
teamvisionscore      float64
goldleadat15           int32
csleadat15             int32
teamdeaths             int64
dtype: object


In [6]:
#Create a function that checks if there are decimal numbers
def has_decimal(arr): 
    return (arr % 1 != 0).any()

In [7]:
#Check for decimal numbers
print(has_decimal(game_player_stats['damagetochampions']))
print(has_decimal(game_player_stats['visionscore']))
print(has_decimal(game_player_stats['earnedgold']))
print(has_decimal(game_player_stats['total cs']))
print(has_decimal(game_player_stats['golddiffat15']))
print(has_decimal(game_player_stats['csdiffat15']))
print(has_decimal(game_player_stats['xpdiffat15']))


False
False
False
False
False
False
False


In [8]:
import sys
print(sys.executable)

g:\Users\Johannes\anaconda3\python.exe


We want to estimate whether a gold lead at 15 has a causal association with the result 
For that we first need to transform so that we have the goldlead instead of golddiffat15

For that we define goldleadat15 as 1 if the golddiffat15 is >= 1000 gold and 0 otherwise 

In [ ]:
game_player_stats['goldleadat15'] = (game_player_stats['golddiffat15'] > 4000).astype(int)
game_player_stats['csleadat15'] = (game_player_stats['csdiffat15'] > 10).astype(int)

print(game_player_stats.dtypes)

gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
teamkills              int64
teamvisionscore      float64
goldleadat15           int32
csleadat15             int32
teamdeaths             int64
propensity_score     float64
dtype: object


we model the scm as seen on the Model notebook (ipad)

In [41]:
from dowhy import CausalModel

#First we create the model for the causal graph
causal_graph = ''' 
digraph {
    teamkills -> goldleadat15;
    teamkills -> result;
    csleadat15 -> goldleadat15;
    csleadat15 -> result;
    teamvisionscore -> teamkills;
    goldleadat15 -> result;
}
'''

model = CausalModel(
    data=game_player_stats,
    treatment='goldleadat15',
    outcome='result',
    graph=causal_graph
)

#We then identify the estimand here the graph is used thus we should end up with
#E_W * E[Y|t, W] where Y is the result and t the treatment here goldleadat15 W is a sufficient blocking set blocking 
#backdoor paths here that would be teamkills, teamdeaths and csleadat15

identified_estimand = model.identify_effect()

#Now we estimate our identified estimand 

estimate = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.propensity_score_matching'
)
print(estimate.value)

0.005772889818500776


In [15]:
from dowhy import CausalModel

#Do a test whether there is a causal impact of tvs on team kills

causal_graph = ''' 
digraph {
    teamvisionscore -> teamkills;
    csdiffat15 -> teamvisionscore;
    csdiffat15 -> teamkills;
    golddiffat15 -> teamvisionscore;
    golddiffat15 -> teamkills;
}
'''

model = CausalModel(
    data = game_player_stats,
    treatment = 'teamvisionscore',
    outcome = 'teamkills',
    graph = causal_graph
)

identified_estimand = model.identify_effect()
#print(identified_estimand)
estimate = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.linear_regression'
)
print(estimate.value)

refutation = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name = 'random_common_cause',
    num_simulations = 20
)
print(refutation)

g:\Users\Johannes\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]


0.01174856727960183


g:\Users\Johannes\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
g:\Users\Johannes\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
g:\Users\Johannes\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consi

Refute: Add a random common cause
Estimated effect:0.01174856727960183
New effect:0.011748557460449139
p value:0.46141033929369013



g:\Users\Johannes\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]


outcome : teamkills
treatment: visionscore

graph: 
tvs -> team_kills
csdiffat15 -> tvs
golddiffat15 -> tvs
